# Overview
Use this notebook to convert OCT 3D volume (stored in .tiff)  to virtual histology.

To get started,
[open this notebook in colab](https://colab.research.google.com/github/WinetraubLab/OCT2Hist-ModelInference/blob/main/run_oct2hist3D.ipynb) and run.


In [1]:
# @title Notebook Inputs { display-mode: "form" }
# @markdown Input Image Path
# Path to the OCT image
oct_input_volume_path = "/content/drive/Shareddrives/Yolab - Current Projects/Aidan/2023 Alopecia Test/AlopeciaTest_Original1.tiff" # @param {type:"string"}

# OCT image's pixel size
microns_per_pixel_z = 1
microns_per_pixel_x = 1
microns_per_pixel_y = 1

# @markdown Cropping Parameters
x0 = 0 # @param {type:"slider", min:0, max:1000, step:10}
z0 = 150 # @param {type:"slider", min:0, max:1000, step:10}

output_tiff_path = "/content/drive/Shareddrives/Yolab - Current Projects/Yonatan/Output.tif" # @param {type:"string"}
section_separation_um = 5 # @param {type:"slider", min:1, max:20, step:1}

min_signal_threshold = 0.46

## Set Up Environment

In [2]:
if not 'setup_env_oct2Hist_usemodel' in globals():
  setup_env_oct2Hist_usemodel = True
  !git clone --recurse-submodules https://github.com/WinetraubLab/OCT2Hist-ModelInference
  !pip install imageio
  %cd OCT2Hist-ModelInference

import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import oct2hist
from utils.show_images import *
from google.colab import drive
from utils.crop import crop
import os
import shutil
from tqdm import tqdm
drive.mount('/content/drive/')

Cloning into 'OCT2Hist-ModelInference'...
remote: Enumerating objects: 651, done.
remote: Counting objects: 100% (147/147), done.
remote: Compressing objects: 100% (146/146), done.
remote: Total 651 (delta 80), reused 3 (delta 1), pack-reused 504
Receiving objects: 100% (651/651), 22.64 MiB | 19.30 MiB/s, done.
Resolving deltas: 100% (337/337), done.
Submodule 'pytorch-CycleGAN-and-pix2pix' (https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix) registered for path 'pytorch-CycleGAN-and-pix2pix'
Cloning into '/content/OCT2Hist-ModelInference/pytorch-CycleGAN-and-pix2pix'...
remote: Enumerating objects: 2513, done.        
remote: Total 2513 (delta 0), reused 0 (delta 0), pack-reused 2513        
Receiving objects: 100% (2513/2513), 8.20 MiB | 9.92 MiB/s, done.
Resolving deltas: 100% (1575/1575), done.
Submodule path 'pytorch-CycleGAN-and-pix2pix': checked out '9f8f61e5a375c2e01c5187d093ce9c2409f409b0'
/content/OCT2Hist-ModelInference
Mounted at /content/drive/


In [3]:
# Set up the network environment
oct2hist.setup_network()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
def merge_tiff_files(input_folder, output_file):
    # Get a list of all TIFF files in the input folder
    tiff_files = [file for file in os.listdir(input_folder) if file.lower().endswith('.png')]

    # Sort the files if needed
    tiff_files.sort()

    # Open all TIFF files
    images = [Image.open(os.path.join(input_folder, file)) for file in tiff_files]

    # Save as multi-page TIFF
    images[0].save(output_file, save_all=True, append_images=images[1:])

## Run
In this step we load the OCT image, then crops it to the right size



In [5]:
# Load OCT image
_, oct_volume = cv2.imreadmulti(oct_input_volume_path)
oct_volume = np.asarray(oct_volume)

# Create a temporary folder for individual scans
tmp_folder = '/content/tmp/'
if os.path.exists(tmp_folder):
    shutil.rmtree(tmp_folder)
    print(f"Folder '{tmp_folder}' deleted.")
os.makedirs(tmp_folder) # Create a new folder

# Loop over entire volume convert and append to outputfile
for i in tqdm(
    range(0,oct_volume.shape[0],int(section_separation_um/microns_per_pixel_y)),
    desc="Converting OCT to Histology", unit="image"):
  # Crop
  oct_image = cv2.cvtColor(oct_volume[i,:,:], cv2.COLOR_GRAY2BGR)
  cropped = crop(oct_image, target_width=1024, target_height=512, x0=x0, z0=z0)

  # Run Inference
  virtual_histology_image, *_ = oct2hist.run_network(
    cropped,
    microns_per_pixel_x = microns_per_pixel_x,
    microns_per_pixel_z = microns_per_pixel_z,
    min_signal_threshold = min_signal_threshold
    )

  # Save to tmp file
  filepath = tmp_folder + "{0:04}.png".format(i)
  bgr_image = cv2.cvtColor(virtual_histology_image, cv2.COLOR_RGB2BGR)
  cv2.imwrite(filepath, bgr_image)

# Combine files to one volume
print('Exporting file to: ' + output_tiff_path)
merge_tiff_files(tmp_folder,output_tiff_path)

Converting OCT to Histology:   0%|          | 0/25 [00:00<?, ?image/s]/content/OCT2Hist-ModelInference/utils/masking.py:86: RuntimeWarning: Mean of empty slice
  m_mean = np.nanmean(filt_img, axis=1)
Converting OCT to Histology: 100%|██████████| 25/25 [01:52<00:00,  4.51s/image]


Exporting file to: /content/drive/Shareddrives/Yolab - Current Projects/Yonatan/Output.tif
